In [1]:
import requests, json, csv

In [8]:
api = {
    'gateway': 'https://manifesto-project.wzb.eu/tools/',
    'key': '?api_key=5bc549b3a55fdfe3f9ed7747c437f3e0'
}

In [9]:
def api_request(api_method, *params):
    parameters = ''
    if params:
        for i in params:
            parameters += ('&' + i)
    request_url = api['gateway'] + api_method + '.json' + api['key'] + parameters
    print(request_url)
    return json.loads(requests.get(request_url).text)

In [34]:
def write_csv(data, name):
    path = 'datasheets/' + name
    with open(path, "w", newline='') as csv_file:
            writer = csv.writer(csv_file, delimiter=',')
            for line in data:
                writer.writerow(line)

In [39]:
api_request('api_list_metadata_versions')

https://manifesto-project.wzb.eu/tools/api_list_metadata_versions?api_key=5bc549b3a55fdfe3f9ed7747c437f3e0


{'versions': ['2015-1',
  '20150603140744',
  '20150603144655',
  '2015-2',
  '20150619183523',
  '20150708095326',
  '20150708102056',
  '2015-3',
  '20150716220651',
  '20150727144348',
  '20150914183516',
  '2015-4',
  '2015-5',
  '20151207125518',
  '20160205170254',
  '20160205171117',
  '20160205184152',
  '2016-1',
  '2016-2',
  '20160520191208',
  '20160524112135',
  '2016-3',
  '20160817112523',
  '20160817115608',
  '20160823133322',
  '2016-4',
  '2016-5',
  '20161128161012',
  '2016-6',
  '20170720161359',
  '2017-1',
  '20170815135004',
  '20171211135550',
  '2017-2']}

In [32]:
api_request('api_list_core_versions')

https://manifesto-project.wzb.eu/tools/api_list_core_versions?api_key=5bc549b3a55fdfe3f9ed7747c437f3e0&key=455&datg


{'datasets': [{'id': 'MPDS2012a',
   'name': 'Manifesto Project Dataset (version 2012a)'},
  {'id': 'MPDS2012b', 'name': 'Manifesto Project Dataset (version 2012b)'},
  {'id': 'MPDS2013a', 'name': 'Manifesto Project Dataset (version 2013a)'},
  {'id': 'MPDS2013b', 'name': 'Manifesto Project Dataset (version 2013b)'},
  {'id': 'MPDS2014a', 'name': 'Manifesto Project Dataset (version 2014a)'},
  {'id': 'MPDS2014b', 'name': 'Manifesto Project Dataset (version 2014b)'},
  {'id': 'MPDS2015a', 'name': 'Manifesto Project Dataset (version 2015a)'},
  {'id': 'MPDS2016a', 'name': 'Manifesto Project Dataset (version 2016a)'},
  {'id': 'MPDS2016b', 'name': 'Manifesto Project Dataset (version 2016b)'},
  {'id': 'MPDS2017a', 'name': 'Manifesto Project Dataset (version 2017a)'},
  {'id': 'MPDS2017b', 'name': 'Manifesto Project Dataset (version 2017b)'}]}

In [46]:
api_request('api_metadata', 'keys[]=62320_199706', 'version=2015-1')

https://manifesto-project.wzb.eu/tools/api_metadata?api_key=5bc549b3a55fdfe3f9ed7747c437f3e0&keys[]=62320_199706&version=2015-1


{'items': [{'party_id': 62320,
   'election_date': '199706',
   'language': 'NA',
   'has_eu_code': 'FALSE',
   'is_primary_doc': 'TRUE',
   'may_contradict_core_dataset': 'FALSE',
   'manifesto_id': '62320_1997',
   'md5sum_text': '29c03f11b740638aaae98be8ef60896d',
   'url_original': None,
   'md5sum_original': None,
   'annotations': False}],
 'missing_items': []}

In [47]:
api_request('api_texts_and_annotations', 'keys=62320_1997', 'version=2015-1')

https://manifesto-project.wzb.eu/tools/api_texts_and_annotations?api_key=5bc549b3a55fdfe3f9ed7747c437f3e0&keys=62320_1997&version=2015-1


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [37]:
write_csv(api_request('api_get_core', 'key=MPDS2017b'), 'core.csv')

https://manifesto-project.wzb.eu/tools/api_get_core?api_key=5bc549b3a55fdfe3f9ed7747c437f3e0&key=MPDS2017b
